In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

import tensorflow as tf
import numpy as np

# Boston Model

In [2]:
X_train = np.load(os.path.join('data', 'boston', 'boston_train_vectors.npy'), allow_pickle=True)
X_test = np.load(os.path.join('data', 'boston', 'boston_test_vectors.npy'), allow_pickle=True)
y_train = np.load(os.path.join('data', 'boston', 'boston_train_labels.npy'), allow_pickle=True)
y_test = np.load(os.path.join('data', 'boston', 'boston_test_labels.npy'), allow_pickle=True)

In [3]:
X_train.shape, y_train.shape

((404, 13), (404,))

In [4]:
tf.random.set_seed(42)
np.random.seed(42)
tf.keras.backend.set_floatx('float64')

model = Sequential()
model.add(Dense(13, activation='sigmoid'))
model.add(Dense(50, activation='sigmoid'))
model.add(Dense(50, activation='sigmoid'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='sgd', metrics=['mae'])

In [5]:
cb = [EarlyStopping(monitor="val_mae", min_delta=0.01, patience=2, verbose=1, 
                    mode="auto", baseline=None, restore_best_weights=True)]

In [6]:
#model.fit(X_train, y_train, batch_size=1, epochs=20, validation_split=0.1, callbacks=cb)

# Using the custom training loop

In [7]:
from loop import TrainingLoop
import keras

Using TensorFlow backend.


In [8]:

optimizer = tf.keras.optimizers.Adam()
loss_function = tf.keras.losses.MeanSquaredError()

batch_size = 1
epochs = 20

train_metrics = tf.keras.metrics.MeanAbsoluteError()
val_metrics = tf.keras.metrics.MeanAbsoluteError()

In [9]:
training = TrainingLoop(model, X_train, y_train, loss_function, optimizer, train_metrics, val_metrics, validation_split=0.1, batch_size=batch_size)

In [10]:
training.train(epochs)

Epoch 1/20	Loss: 72.283	Metrics: 15.168: 	Validation metrics: 10.229: 	100% | 363/363 [00:01<00:00, 323.23it/s]
Epoch 2/20	Loss: 8.6634	Metrics: 8.5690: 	Validation metrics: 7.4581: 	100% | 363/363 [00:00<00:00, 588.99it/s]
Epoch 3/20	Loss: 0.0131	Metrics: 6.8181: 	Validation metrics: 7.4331: 	100% | 363/363 [00:00<00:00, 550.71it/s]
Epoch 4/20	Loss: 0.9017	Metrics: 6.5793: 	Validation metrics: 7.3604: 	100% | 363/363 [00:00<00:00, 588.35it/s]
Epoch 5/20	Loss: 2.8709	Metrics: 6.1676: 	Validation metrics: 6.5593: 	100% | 363/363 [00:00<00:00, 601.87it/s]
Epoch 6/20	Loss: 8.0239	Metrics: 5.5194: 	Validation metrics: 5.6504: 	100% | 363/363 [00:00<00:00, 615.61it/s]
Epoch 7/20	Loss: 15.298	Metrics: 5.0236: 	Validation metrics: 5.0925: 	100% | 363/363 [00:00<00:00, 605.88it/s]
Epoch 8/20	Loss: 21.717	Metrics: 4.7738: 	Validation metrics: 4.8580: 	100% | 363/363 [00:00<00:00, 577.68it/s]
Epoch 9/20	Loss: 25.164	Metrics: 4.6436: 	Validation metrics: 4.6387: 	100% | 363/363 [00:00<00:00, 598.

In [23]:
res = model.evaluate(X_test, y_test)
print(np.sqrt(res[0]), res[1])

4/4 [==============================] - 0s 859us/step - loss: 18.2496 - mae: 3.0743
4.271956026494088 3.0742857456207275
